In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torcheval.metrics.functional import multiclass_accuracy
from torch.utils.data import TensorDataset, DataLoader, Subset
import random

In [ ]:
rating_df = pd.read_csv('data/MovieLens/rating.csv')
rating_df.head()

In [ ]:
# Let's make it multi-class classification
rating_df['normalized_rating'] = rating_df.rating
rating_values = np.array(list(sorted(set(rating_df.rating))))
# num_classes = len(set(rating_df.rating))
rating_df['rating_class'] = (rating_df.rating * 2).astype(int)

In [ ]:
set(rating_df.rating_class)

In [ ]:
subset_ratio = 0.003
rows = rating_df.userId - min(rating_df.userId) # convert into index starting at 0
cols = rating_df.movieId - min(rating_df.movieId) # convert into index starting at 0
user_movie_coo_tensor = torch.sparse_coo_tensor([rows, cols], rating_df.rating_class.to_numpy(), dtype=torch.float)
user_movie_dataset = TensorDataset(user_movie_coo_tensor.to_dense())
user_movie_dataset_length = len(user_movie_dataset)
user_movie_subset_sample_idx = random.sample(range(user_movie_dataset_length), int(user_movie_dataset_length * subset_ratio))
user_movie_subset = Subset(user_movie_dataset, user_movie_subset_sample_idx)
user_movie_dataloader = DataLoader(user_movie_subset, batch_size=64, shuffle=True)

In [ ]:
class MovieLensVAE(nn.Module):
    def __init__(self, input_dim, num_classes=6, hidden_dim=600, latent_dim=200):
        super().__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim, bias=True),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU()
        )
        self.mu = nn.Linear(hidden_dim, latent_dim, bias=True)
        self.logvar = nn.Linear(hidden_dim, latent_dim, bias=True)
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim, bias=True),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, input_dim * num_classes, bias=True)
        )
    
    def forward(self, x):
        latent = self.encoder(x)
        mu = self.mu(latent)
        logvar = self.logvar(latent)
        z = self.reparameterization(mu, logvar)
        x_output = self.decoder(z)
        logits = x_output.view(-1, self.num_classes, self.input_dim)
        return logits, mu, logvar

    def reparameterization(self, mu, logvar):
        std = torch.exp(logvar/2)
        e = torch.randn_like(std)
        return mu + std * e

In [ ]:
# Minimize cross entropy reconstruction_loss + KL Divergence
def loss_func(x, vae_logits, mu, logvar):
    # Reconstruction Loss
    # input = batch_size x C x movie
    # target = batch_size x movie
    reconstruction_loss = F.cross_entropy(vae_logits, x.long())
    # KL Divergence between Q(z|X) and N(0, 1)
    KLD = 0.5 * torch.mean(mu.pow(2) + logvar.exp() - logvar - 1)
    return reconstruction_loss + KLD

In [ ]:
input_dim = max(cols) + 1
num_classes = len(set(rating_df.rating_class)) + 1
movielens_var = MovieLensVAE(input_dim, num_classes, hidden_dim=20, latent_dim=5)
optimizer = optim.Adam(movielens_var.parameters(), lr=1e-3)
epochs = 10

cur_mu, cur_logvar = 0, 0
for epoch in range(epochs):
    loss_err = 0
    for i, dl in enumerate(user_movie_dataloader):
        x_output, mu, logvar = movielens_var(dl[0])
        cur_mu, cur_logvar = mu, logvar
        loss = loss_func(dl[0], x_output, mu, logvar)
        loss_err += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        print(f"Epoch={epoch}, i={i}: Error={loss_err}")

In [ ]:
def predict(model, x):
    # Set the model setting to eval() mode
    model.eval()
    
    # Convert x to tensor if needed
    if not isinstance(x, torch.Tensor):
        x = torch.tensor(x, type=torch.float)
    
    with torch.no_grad():
        logits, _, _ = model(x)
        x_output = torch.softmax(logits, dim=-1)
        x_output = x_output.view(len(x), -1, num_classes)
        x_output = torch.max(x_output, dim=-1)[1]
    
    return x_output

In [ ]:
for x in user_movie_dataloader:
    predicted_x = predict(movielens_var, x[0])
    acc = 0
    for batch in range(64):
        acc += multiclass_accuracy(x[0][i], predicted_x[i])
    print(acc / 64 * 100)
    break